In [ ]:
# load transcripts and extract features for all chunks (IPUs)
import sys 
sys.path.append('../python/')
import cfg
import db
import fio
import vad_asr

In [ ]:
db.connect()
vad_asr.load_transcriptions()
db.commit()
db.close()

In [ ]:
db.connect()
for _, ses_id, _, _, ((_, pA, fnA), (_, pB, fnB)) \
in db.find_sessions(3, '>='):
    for a_or_b, path, fname in [('A', pA, fnA), ('B', pB, fnB)]:
        if fname:
            for chu_id, words, start, end \
            in db.find_chunks(ses_id, a_or_b):
                features = fio.extract_features(
                    path, fname, ses_id, chu_id, words, start, end)
                db.set_features(chu_id, features)
        db.commit()
    print(ses_id, 'done!')
# run cleanup script
db.executescript(cfg.SQL_PATH, cfg.SQL_CU_FNAME)
db.commit()
db.close()